In [1]:
# imports 
import emp_preprocessing as ep
import emp_fake_data as ef
import emp_metrics as em
import networkx as nx
import functions as fct
import numpy as np
import pandas as pd
import parameters as par
from graphics import ClassGraphics

# parameters
path_results = "./results/empirical_results/"
Graphics = ClassGraphics(None,plot_period=1000)

# 1 data preprocessing

In [2]:
# load fake data (exposure)
df_exposures = ef.get_df_exposures(lines=int(1e5))
dic_obs_matrix_reverse_repo = ep.build_from_exposures(df_exposures=df_exposures,path=f"{path_results}data/")

100%|██████████| 100000/100000 [00:33<00:00, 3025.30it/s]


In [3]:
# pickle (faster)
dic_obs_matrix_reverse_repo = ep.load_dic_obs_matrix_reverse_repo(path=f"{path_results}data/")

# 2 metrics measurement

In [4]:
# get aggregated adjency matrices
dic_arr_binary_adj = ep.build_rolling_binary_adj(dic_obs_matrix_reverse_repo=dic_obs_matrix_reverse_repo,path=f"{path_results}data/")

In [5]:
# pickle (faster)
dic_arr_binary_adj = ep.load_dic_arr_binary_adj(path=f"{path_results}data/")

## 2.1 jaccard

In [6]:
days = list(dic_obs_matrix_reverse_repo.keys())

# get jaccard
df_jaccard = em.get_jaccard(dic_arr_binary_adj, days, path=path_results)
Graphics.plot_trajectory(
            df=df_jaccard,
            cols=[
                f"jaccard index-{agg_period}" for agg_period in par.agg_periods
            ],
            title="Jaccard index",
            file_name=f"{path_results}jaccard_index.pdf")

get jaccard


100%|██████████| 20833/20833 [00:04<00:00, 4362.85it/s]


## 2.2 density

In [7]:
days = list(dic_obs_matrix_reverse_repo.keys())

# get density
df_density = em.get_density(dic_arr_binary_adj, days, path=path_results)
Graphics.plot_trajectory(
            df=df_density,
            cols=[
                f"network density-{agg_period}"
                for agg_period in par.agg_periods
            ],
            title="Network density",
            file_name=f"{path_results}network_density.pdf",
        )

get density


100%|██████████| 20833/20833 [00:04<00:00, 4469.74it/s]


## 2.3 degree distribution 

In [8]:
days = list(dic_obs_matrix_reverse_repo.keys())

# get degree distribution
dic_in_degree, dic_out_degree, dic_degree = em.get_degree_distribution(dic_arr_binary_adj, path=f"{path_results}degree_distribution/")
Graphics.plot_degree_distribution(dic_in_degree,
    dic_out_degree,
    days,
    f"{path_results}degree_distribution/")


get degree distribution


 50%|█████     | 10481/20834 [00:05<00:05, 2010.62it/s]

## 2.4 av. degree

In [ ]:
days = list(dic_obs_matrix_reverse_repo.keys())

# get av. degree
df_av_degree = em.get_av_degree(dic_degree, days, path=path_results)
Graphics.plot_trajectory(
            df=df_av_degree,
            cols=[
                f"av. degree-{agg_period}" for agg_period in par.agg_periods
            ],
            title="Av. degree",
            file_name=f"{path_results}average_degree.pdf",
        )

get av. degree


100%|██████████| 2083/2083 [00:00<00:00, 5447.00it/s]


## 2.5 degree per asset

In [ ]:
dic_obs_matrix_reverse_repo["2020-01-01"].shape

(20, 20)

In [ ]:
# build finrep data @NA: add here a read csv of the data with the total assets
df_finrep = ef.get_df_finrep()
days = list(dic_obs_matrix_reverse_repo.keys())
bank_ids = list(dic_obs_matrix_reverse_repo.values())[0].index

# select the days and banks (common between the 2 lists)
finrep_days = list(df_finrep["date"])
finrep_bank_ids = list(df_finrep["lei"])
plot_days = fct.list_intersection(days, finrep_days)

# build arr_total_assets
arr_total_assets = em.build_arr_total_assets(df_finrep, finrep_bank_ids,f"{path_results}degree_per_asset/")

# plot
Graphics.plot_degree_per_asset(arr_total_assets,dic_degree,bank_ids,days,f"{path_results}degree_per_asset/",plot_days=plot_days,finrep_days=finrep_days, finrep_bank_ids=finrep_bank_ids)

ValueError: Length of values (10) does not match length of index (20)

## 2.6 core periphery

In [ ]:
arr_obs_matrix_reverse_repo = ep.convert_dic_to_array(dic_obs_matrix_reverse_repo)
days = list(dic_obs_matrix_reverse_repo.keys())

# run cp tests
Graphics.plot_cp_test(dic_arr_binary_adj, algos=par.cp_algos,days=days, path_results=path_results, opt_agg=True)
Graphics.plot_cp_test(arr_obs_matrix_reverse_repo, algos=par.cp_algos, days=days, path_results=path_results, opt_agg=False)

# 3 tools

## 3.1 examples of density / histogram in log - log

### pareto law

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Generate random data
x = np.random.pareto(3,size=100000)

# Compute the absolute value and sign of the data
abs_x = np.abs(x)
sign_x = np.sign(x)

# Set up the figure
fig, ax = plt.subplots(figsize=(8, 4))

# Plot the KDE on a log scale with the transformed data
sns.kdeplot(x.clip(0.5,5), ax=ax, cut=0, fill=True)
ax.set_xscale('log')

# Set the y-axis to log scale
ax.set_yscale('log')

# Set axis labels
ax.set_xlabel('value')
ax.set_ylabel('density')

# Show the plot
plt.show()

### student law

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Generate random data
x = np.random.standard_t(1,size=100000)

# Compute the absolute value and sign of the data
abs_x = np.abs(x)
sign_x = np.sign(x)

# Set up the figure
fig, ax = plt.subplots(figsize=(8, 4))

# Plot the KDE on a log scale with the transformed data
sns.kdeplot(x.clip(-10,10), ax=ax, fill=True, cut=0)
ax.set_xscale('symlog', linthresh=0.1)

# Set the y-axis to log scale
ax.set_yscale('log')

# Set axis labels
ax.set_xlabel('Value')
ax.set_ylabel('Density')

# Show the plot
plt.show()

## 3.2 multiple histogram on the same plot

In [ ]:
from matplotlib import pyplot as plt

fig, (ax1,ax2) = plt.subplots(1,2)
pos = 0
width = 0.2
for  i in range(10):
    binary_adj = np.random.randint(0,2,(10,10))
    bank_network = nx.from_numpy_array(
                binary_adj,
                parallel_edges=False,
                create_using=nx.DiGraph,
            )  # first define a networkx object.
    ar_in_degree = np.array(bank_network.in_degree())[:,1]
    hist = np.unique(ar_in_degree, return_counts=True)
    ax1.bar(hist[0] + pos, hist[1], width=0.01)
    ax2.bar(hist[0] + pos, hist[1], width=0.01)
    pos = pos +width +0.1



## 3.3 power low fitting

In [ ]:
# fitting
import powerlaw
import numpy as np
data = np.random.pareto(3,size=1000) # data can be list or numpy array
fit = powerlaw.Fit(data)
print(fit.power_law.alpha)
print(fit.power_law.xmin)
R, p = fit.distribution_compare('power_law', 'exponential')
fig2 = fit.plot_ccdf(color = "r", linewidth = 2)
fit.power_law.plot_ccdf(color = "r", linestyle = "--", ax = fig2)
fit.lognormal.plot_ccdf(ax=fig2)
fit.exponential.plot_ccdf(ax=fig2)